# Semantic Search on PDF Documents

This example shows how to use KDB.AI to run semantic search on unstructured text documents.

Semantic search allows users to perform searches based on the meaning or similarity of the data rather than exact matches. It works by converting the query into a vector representation and then finding similar vectors in the database. This way, even if the query and the data in the database are not identical, the system can identify and retrieve the most relevant results based on their semantic meaning.

### Aim
In this tutorial, we'll walk you through the process of performing semantic search on documents, taking PDFs as example, using KDB.AI as the vector store. We will cover the following topics:

0. Load PDF Data
1. Create Sentence Vector Embeddings
2. Store Embeddings in KDB.AI
3. Search For Similar Sentences To A Target Sentence
4. Delete the KDB.AI Table

---

## 0. Setup

### Install dependencies

In order to successfully run this sample, note the following steps depending on where you are running this notebook:

-***Run Locally / Private Environment:*** The [Setup](https://github.com/KxSystems/kdbai-samples/blob/main/README.md#setup) steps in the repository's `README.md` will guide you on prerequisites and how to run this with Jupyter.


-***Colab / Hosted Environment:*** Open this notebook in Colab and run through the cells.

### Set Environment Variables

In [ ]:
!pip install kdbai_client

In [ ]:
!pip install pypdf sentence_transformers

In [ ]:
### !!! Only run this cell if you need to download the data into your environment, for example in Colab
### This downloads research paper pdf into your environment
!mkdir -pv ./data
!wget -P ./data https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/document_search/data/research_paper.pdf

In [4]:
import os

In [5]:
### ignore tensorflow warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

### Import Packages

In [ ]:
# load data
import pypdf
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
# embeddings
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

In [8]:
# vector DB
import os
import kdbai_client as kdbai
from getpass import getpass
import time

### Configure Console

In [9]:
pd.set_option("display.max_colwidth", 300)

### Define Helper Functions

In [10]:
def show_df(df: pd.DataFrame) -> pd.DataFrame:
    print(df.shape)
    return df.head()

## 1. Load PDF Data

### Read Text From PDF Document

We leverage the power of PyPDF2 for PDF processing and `nltk` for advanced natural language processing. The code below extracts content from each page of the PDF and processes it to identify sentences.

The PDF we are using is [this research paper](https://arxiv.org/pdf/2308.05801.pdf) presenting information on the formation of Interstellar Objects in the Milky Way.

In [11]:
# Read PDF file
with open("data/research_paper.pdf", "rb") as pdf_file:
    pdf_pages = pypdf.PdfReader(pdf_file).pages
    page_list = [page.extract_text() for page in pdf_pages]

In [12]:
# Concatenate text from each page
full_pdf_text = "".join(page_list)

### Split The Text Into Sentences

<div class="alert alert-block alert-warning">
    <b>Note: </b>
    Before running the following line of code, please ensure that you have installed the English sentence tokenizer as stated in the `README.md` file in this repository.
</div>

In [13]:
# Split the PDF into sentences
pdf_sentences = sent_tokenize(full_pdf_text)
len(pdf_sentences)

591

In [14]:
pdf_sentences[0]

'Draft version August 14, 2023\nTypeset using L ATEX default style in AASTeX631\nThe Galactic Interstellar Object Population: A Framework for Prediction and Inference\nMatthew J. Hopkins\n ,1Chris Lintott\n ,1Michele T. Bannister\n ,2J.'

## 2. Create Sentence Vector Embeddings

Next, we use the Sentence Transformers library to create embeddings for our collection of sentences.

### Selecting a Sentence Transformer Model

There are 100+ of different types of Sentence Transformers models available - see [HuggingFace](https://huggingface.co/sentence-transformers) for the full list. The diversity among these primarily stems from variations in their training data. Selecting the ideal model for your needs involves matching the domain and task closely, while also considering the benefits of incorporating larger datasets to enhance scale.

This tutorial will use the `all-MiniLM-L6-v2` pre-trained model. This embedding model can create sentence and document embeddings that can be used for a wide variety of tasks including semantic search which makes it a good choice for our needs.

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

### Generate Sentence Embeddings Using This Model

We prepare embeddings by applying the sentence transformer model to our sentences to encode them. The we do some transformation to get this into DataFrame which is the format accepted by KDB.AI.

In [16]:
# Create embeddings
embeddings_list = model.encode(np.array(pdf_sentences)).tolist()
embeddings_df = pd.DataFrame({"vectors": embeddings_list, "sentences": pdf_sentences})

In [17]:
show_df(embeddings_df)

(591, 2)


,vectors,sentences
0,"[-0.059804268181324005, -0.09221810102462769, 0.058069996535778046, 0.06884294003248215, -0.0030452304054051638, 0.007304240483790636, -0.028959110379219055, 0.0595787838101387, 0.017943743616342545, 0.043501030653715134, 0.005012953653931618, -0.07875007390975952, -0.02570340223610401, -0.04147...","Draft version August 14, 2023\nTypeset using L ATEX default style in AASTeX631\nThe Galactic Interstellar Object Population: A Framework for Prediction and Inference\nMatthew J. Hopkins\n ,1Chris Lintott\n ,1Michele T. Bannister\n ,2J."
1,"[-0.08154530823230743, -0.11342314630746841, 0.08425560593605042, 0.08849422633647919, 0.024723634123802185, -0.08773960173130035, -0.06128991022706032, 0.0106121264398098, 0.06387822329998016, 0.021902449429035187, -0.033225465565919876, -0.09321605414152145, -0.04425090178847313, -0.0568475760...","Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Forbes\n2\n1Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK\n2School of Physical and Chemical Sciences—Te Kura Mat¯ u, University of Canterbury, Private Bag 4800, Christchurch 8140, New Zealand\n3Just ..."
2,"[-0.07799109816551208, -0.08398666232824326, 0.02723710611462593, 0.09858055412769318, 0.010515090078115463, -0.01670389622449875, -0.055797528475522995, 0.027984360232949257, 0.03926650434732437, 0.03248238563537598, 0.03837994113564491, -0.12464361637830734, -0.012645130045711994, -0.018288377...","We define a novel framework: firstly to predict\nthe properties of this Galactic ISO population by combining models of processes across planetary\nand galactic scales, and secondly to make inferences about the processes modelled, by comparing the\npredicted population to what is observed."
3,"[-0.05338575318455696, -0.037147365510463715, 0.07784582674503326, 0.05165949463844299, -0.07674478739500046, -0.007766904775053263, 0.008464669808745384, 0.04289115220308304, 0.04356798157095909, 0.06012833118438721, 0.04407043382525444, -0.10177762806415558, 0.030359704047441483, -0.0340371690...",We predict the spatial and compositional distribution of the\nGalaxy’s population of ISOs by modelling the Galactic stellar population with data from the APOGEE\nsurvey and combining this with a protoplanetary disk chemistry model.
4,"[-0.06326982378959656, -0.01720457151532173, 0.047737207263708115, 0.08325360715389252, 0.03296627104282379, -0.0660020187497139, 0.0015915816184133291, 0.003246008651331067, 0.03456269949674606, -0.020524047315120697, 0.041225481778383255, -0.15274153649806976, -0.011771060526371002, 0.07314326...","Selecting ISO water mass\nfraction as an example observable quantity, we evaluate its distribution both at the position of the Sun\nand averaged over the Galactic disk; our prediction for the Solar neighbourhood is compatible with the\ninferred water mass fraction of 2I/Borisov."


It is important to note the dimension of our embeddings is 384. This will need to match the dimensions we set in the KDB.AI index in the next step. We can easily check this using `len` to count elements in our vector.

In [18]:
len(embeddings_df["vectors"][0])

384

## 3. Store Embeddings in KDB.AI

With the embeddings created, we need to store them in a vector database to enable efficient searching.

### Connect to KDB.AI Server

To use KDB.AI Server, you will need download and run your own container.
To do this, you will first need to sign up for free [here](https://trykdb.kx.com/kdbaiserver/signup/).

You will receive an email with the required license file and bearer token needed to download your instance.
Follow instructions in the signup email to get your session up and running.

Once the [setup steps](https://code.kx.com/kdbai/gettingStarted/kdb-ai-server-setup.html) are complete you can then connect to your KDB.AI Server session using `kdbai.Session` and passing your local endpoint.

In [ ]:
#Set up KDB.AI server endpoint 
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else "http://localhost:8082"
)


#connect to KDB.AI Server, default mode is qipc
session = kdbai.Session(endpoint=KDBAI_ENDPOINT)

### Verify Defined Databases

We can check our connection using the `session.databases()` function.
This will return a list of all the databases we have defined in our vector database thus far.
This should return a "default" database along with any other databases you have already created.

In [30]:
session.databases()

[KDBAI database "default"]

### Create a Database called "myDatabase"

In [29]:
# ensure no database called "myDatabase" exists
try:
    session.database("myDatabase").drop()
except kdbai.KDBAIException:
    pass

In [31]:
# Create the database
db = session.create_database("myDatabase")

### Define Vector DB Table Schema

The next step is to define a schema for our KDB.AI table where we will store our embeddings. Our table will have two columns: sentences and vectors.


In [50]:
pdf_schema = [
        {"name": "sentences", "type": "str"},
        {
            "name": "vectors",
            "type": "float64s",
        },
    ]

### Define the indexes
We will define our dimensionality, similarity metric and index type with the vectorIndex attribute. For this example we chose:

- type = hnsw : HNSW enhances efficiency while maintaining accuracy. You have the choice of using other indexes like, qHNSW, and IVFPQ, qFlat or a Flat index here, as with metrics the one you chose depends your data and your overall performance requirements.
- name = hnsw_index : this is a custom name you give your index.
- column = vectors : this is the column where the embeddings are stored.
#### params:
- dims = 384 : In the next section, we generate embeddings that are 384-dimensional to match this. The number of dimensions should mirror the output dimensions of your embedding model.
- metric = L2 : We chose Euclidean Distance. You have the choice of using other metrics here like IP/Inner Product and CS/Cosine Similarity and the one you chose depends on the specific context and nature of your data.
!Note, it is possible to define multiple indexes within a table!

In [51]:
# Define the index
indexes = [
    {
        'type': 'hnsw',
        'name': 'hnsw_index',
        'column': 'vectors',
        'params': {'dims': 384, 'metric': "L2"},
    },
]

### Create Vector DB Table

Use the KDB.AI `create_table` function to create a table that matches the defined schema in the vector database.

In [52]:
# First ensure the table does not already exist
try:
    db.table("pdf").drop()
except kdbai.KDBAIException:
    pass

In [53]:
table = db.create_table(table="pdf", schema=pdf_schema, indexes=indexes)

We can use `query` to see our table exists but is empty.

In [54]:
table.query()

,sentences,vectors


### Add Embedded Data to KDB.AI Table

In [55]:
table.insert(embeddings_df)

{'rowsInserted': 591}

### Verify Data Has Been Inserted

Running `table.query()` should show us that data has been added.

In [56]:
show_df(table.query())

(591, 2)


,vectors,sentences
0,"[-0.059804268181324005, -0.09221810102462769, 0.058069996535778046, 0.06884294003248215, -0.0030452304054051638, 0.007304240483790636, -0.028959110379219055, 0.0595787838101387, 0.017943743616342545, 0.043501030653715134, 0.005012953653931618, -0.07875007390975952, -0.02570340223610401, -0.04147...","Draft version August 14, 2023\nTypeset using L ATEX default style in AASTeX631\nThe Galactic Interstellar Object Population: A Framework for Prediction and Inference\nMatthew J. Hopkins\n ,1Chris Lintott\n ,1Michele T. Bannister\n ,2J."
1,"[-0.08154530823230743, -0.11342314630746841, 0.08425560593605042, 0.08849422633647919, 0.024723634123802185, -0.08773960173130035, -0.06128991022706032, 0.0106121264398098, 0.06387822329998016, 0.021902449429035187, -0.033225465565919876, -0.09321605414152145, -0.04425090178847313, -0.0568475760...","Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Forbes\n2\n1Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK\n2School of Physical and Chemical Sciences—Te Kura Mat¯ u, University of Canterbury, Private Bag 4800, Christchurch 8140, New Zealand\n3Just ..."
2,"[-0.07799109816551208, -0.08398666232824326, 0.02723710611462593, 0.09858055412769318, 0.010515090078115463, -0.01670389622449875, -0.055797528475522995, 0.027984360232949257, 0.03926650434732437, 0.03248238563537598, 0.03837994113564491, -0.12464361637830734, -0.012645130045711994, -0.018288377...","We define a novel framework: firstly to predict\nthe properties of this Galactic ISO population by combining models of processes across planetary\nand galactic scales, and secondly to make inferences about the processes modelled, by comparing the\npredicted population to what is observed."
3,"[-0.05338575318455696, -0.037147365510463715, 0.07784582674503326, 0.05165949463844299, -0.07674478739500046, -0.007766904775053263, 0.008464669808745384, 0.04289115220308304, 0.04356798157095909, 0.06012833118438721, 0.04407043382525444, -0.10177762806415558, 0.030359704047441483, -0.0340371690...",We predict the spatial and compositional distribution of the\nGalaxy’s population of ISOs by modelling the Galactic stellar population with data from the APOGEE\nsurvey and combining this with a protoplanetary disk chemistry model.
4,"[-0.06326982378959656, -0.01720457151532173, 0.047737207263708115, 0.08325360715389252, 0.03296627104282379, -0.0660020187497139, 0.0015915816184133291, 0.003246008651331067, 0.03456269949674606, -0.020524047315120697, 0.041225481778383255, -0.15274153649806976, -0.011771060526371002, 0.07314326...","Selecting ISO water mass\nfraction as an example observable quantity, we evaluate its distribution both at the position of the Sun\nand averaged over the Galactic disk; our prediction for the Solar neighbourhood is compatible with the\ninferred water mass fraction of 2I/Borisov."


## 4. Search For Similar Sentences To A Target Sentence

Now that the embeddings are stored in KDB.AI, we can perform semantic search using `search`.

### Search 1

First, we embed our search term using the Sentence Transformer model as before. Then we search our index to return the three most similar vectors.

In [57]:
search_term1 = "number of interstellar objects in the milky way"

In [58]:
encoded_search_term1 = model.encode(search_term1).tolist()

In [59]:
results1 = table.search(vectors={"hnsw_index":[encoded_search_term1]}, n=3)
results1[0]

,__nn_distance,vectors,sentences
0,0.678681,"[-0.08154530823230743, -0.11342314630746841, 0.08425560593605042, 0.08849422633647919, 0.024723634123802185, -0.08773960173130035, -0.06128991022706032, 0.0106121264398098, 0.06387822329998016, 0.021902449429035187, -0.033225465565919876, -0.09321605414152145, -0.04425090178847313, -0.0568475760...","Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Forbes\n2\n1Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK\n2School of Physical and Chemical Sciences—Te Kura Mat¯ u, University of Canterbury, Private Bag 4800, Christchurch 8140, New Zealand\n3Just ..."
1,0.665591,"[-0.0868825614452362, -0.023635495454072952, 0.06744848191738129, 0.08018778264522552, -0.05974714457988739, -0.03222731873393059, -0.008770040236413479, 0.04886192828416824, -0.05181043595075607, 0.017594708129763603, 0.024309009313583374, -0.09147438406944275, 0.038199249655008316, -0.06181899...","In this work, we develop\nthis method and apply it to the stellar population of the Milky Way, estimated with data from the APOGEE survey, to\npredict a broader set of properties of our own Galaxy’s population of interstellar objects."
2,0.629392,"[-0.07292614132165909, 0.021450141444802284, 0.01660231687128544, 0.05125197023153305, -0.017564907670021057, -0.08743879944086075, -0.020641718059778214, 0.037198327481746674, -0.024873124435544014, 0.039150986820459366, 0.043217096477746964, -0.1320355087518692, -0.033658064901828766, -0.06451...","Keywords: Interstellar objects (52), Small Solar System bodies(1469), Galaxy Evolution (594)\n1.INTRODUCTION\n1I/‘Oumuamua (Meech et al."


The results returned from `table.search` show the closest matches along with value of nearest neighbor distances `nn_distance`.
We can see these sentences do reference our search term 'number of interstellar objects in the milky way' in some way.

### Search 2

Let's try another search term.

In [61]:
search_term2 = "how does planet formation occur"

In [62]:
encoded_search_term2 = model.encode(search_term2).tolist()

In [63]:
results2 = table.search(vectors={"hnsw_index":[encoded_search_term2]}, n=3)
results2[0]

,__nn_distance,vectors,sentences
0,0.546382,"[-0.07665147632360458, -0.06582025438547134, 0.034305740147829056, 0.026705816388130188, 0.07752171903848648, -0.05098922178149223, 0.007996230386197567, 0.023463979363441467, 0.09635236114263535, 0.05890350416302681, -0.009348639287054539, -0.04947573319077492, 0.04072212800383568, -0.086648009...","The pop-\nulation’s dominant dynamical formation mechanisms would preferentially harvest more distant, ice-rich planetesimals\nfrom the disks of the source systems."
1,0.533592,"[-0.011731946840882301, -0.06267537921667099, 0.08392807841300964, -0.036336500197649, -0.0021244140807539225, -0.05082397535443306, -0.00048589325160719454, -0.02759421430528164, 0.13681785762310028, 0.06662701815366745, -0.02651246450841427, -0.019386690109968185, 0.0160849429666996, -0.083458...","A protoplanetary disk has to first order the same composition as the star it forms around,\nsince they both form from the same molecular cloud core."
2,0.522974,"[-0.0878974199295044, -0.053996648639440536, 0.08170221745967865, 0.05057608336210251, -0.001267113140784204, -0.051809072494506836, -0.03642294555902481, 0.014396563172340393, 0.10822276026010513, 0.043686095625162125, -0.07996439933776855, -0.07062527537345886, 0.05373870208859444, -0.06110462...","While in reality, stars will each produce a distribution of ISOs that\nformed at different positions in their protoplanetary disk and thus have a range of compositions, this simplification\nof only modelling planetesimals which form exterior to the water ice line is justified by the proportional..."


Again, we can see these sentences do reference our search term 'how does planet formation occur' in some way.

## 5. Delete the KDB.AI Database & Table

Once finished with the database & table, it is best practice to drop it.

In [64]:
table.drop()
db.drop()

True

## Take Our Survey

We hope you found this sample helpful! Your feedback is important to us, and we would appreciate it if you could take a moment to fill out our brief survey. Your input helps us improve our content.

[**Take the Survey**](https://delighted.com/t/ezUwZ88a)